# SHAP 可解釋性分析示範

本筆記本展示如何使用SHAP對中文網路霸凌偵測模型進行可解釋性分析

## 功能展示
- Force plots (局部解釋)
- Waterfall plots (層次化解釋)
- Text plots (文本級解釋)
- Summary plots (全局解釋)
- 與IG結果對比
- 誤判分析

In [ ]:
# 導入必要的庫
import sys
import os
import warnings
from pathlib import Path

# 添加專案路徑
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root / "src"))

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings('ignore')

print("環境設定完成")
print(f"專案根目錄: {project_root}")
print(f"Python版本: {sys.version}")
print(f"PyTorch版本: {torch.__version__}")
print(f"GPU可用: {torch.cuda.is_available()}")

In [ ]:
# 導入專案模組
from cyberpuppy.models.improved_detector import ImprovedDetector, create_improved_config
from cyberpuppy.explain.shap_explainer import (
    SHAPExplainer, SHAPVisualizer, MisclassificationAnalyzer,
    compare_ig_shap_explanations
)
from cyberpuppy.explain.ig import IntegratedGradientsExplainer

print("模組導入成功")

## 1. 載入訓練好的模型

In [ ]:
# 載入模型配置
config = create_improved_config()
print(f"模型配置: {config.model_name}")

# 初始化模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedDetector(config)
model.to(device)
model.eval()

print(f"模型已載入到設備: {device}")
print(f"模型參數量: {sum(p.numel() for p in model.parameters()):,}")

# 注意：實際使用時需要載入訓練好的權重
# model.load_state_dict(torch.load('path/to/trained_model.pth'))
print("\n⚠️ 注意：這是未訓練的模型，實際使用時需要載入訓練好的權重")

## 2. 初始化SHAP解釋器

In [ ]:
# 初始化SHAP解釋器
shap_explainer = SHAPExplainer(model, device)
shap_visualizer = SHAPVisualizer(shap_explainer)

print("SHAP解釋器初始化完成")

# 同時初始化IG解釋器用於對比
try:
    ig_explainer = IntegratedGradientsExplainer(model, device)
    print("IG解釋器初始化完成")
    ig_available = True
except Exception as e:
    print(f"IG解釋器初始化失敗: {e}")
    ig_available = False

## 3. 準備測試文本

In [ ]:
# 測試文本集合
test_texts = [
    "你這個垃圾，快去死吧！",  # 明顯毒性/威脅
    "今天天氣真好，心情很愉快",  # 正面情緒
    "這個政策實在是太爛了",  # 輕微負面
    "你再這樣我就要告訴老師了",  # 輕微威脅
    "謝謝你幫助我完成作業",  # 正面感謝
    "你們這些笨蛋什麼都不懂",  # 輕微霸凌
    "我覺得這個想法不太好",  # 中性意見
    "去你的，別煩我！",  # 粗俗語言
    "大家一起努力完成這個專案",  # 正面合作
    "你這個白癡真是無可救藥"  # 人身攻擊
]

print(f"準備了 {len(test_texts)} 個測試文本")
for i, text in enumerate(test_texts, 1):
    print(f"{i:2d}. {text}")

## 4. SHAP解釋示範

In [ ]:
# 選擇一個文本進行詳細分析
sample_text = test_texts[0]  # "你這個垃圾，快去死吧！"
print(f"分析文本: {sample_text}")

# 獲取SHAP解釋
print("\n計算SHAP值...")
shap_result = shap_explainer.explain_text(sample_text, max_evals=500)

# 顯示預測結果
print("\n=== 預測結果 ===")
print(f"毒性預測: {shap_result.toxicity_pred} (置信度: {shap_result.toxicity_prob:.3f})")
print(f"霸凌預測: {shap_result.bullying_pred} (置信度: {shap_result.bullying_prob:.3f})")
print(f"角色預測: {shap_result.role_pred} (置信度: {shap_result.role_prob:.3f})")
print(f"情緒預測: {shap_result.emotion_pred} (置信度: {shap_result.emotion_prob:.3f})")

# 顯示特徵重要性
print("\n=== 特徵重要性 ===")
for task, importance in shap_result.feature_importance.items():
    print(f"{task}: {importance:.4f}")

print("\nSHAP解釋完成")

## 5. Force Plot 可視化

In [ ]:
# 創建毒性分類的Force Plot
print("創建Force Plot - 毒性分類")
try:
    plt.figure(figsize=(15, 8))
    shap_visualizer.create_force_plot(
        shap_result, 
        task="toxicity",
        save_path="../reports/shap_force_plot_toxicity.png"
    )
    plt.show()
except Exception as e:
    print(f"Force plot創建失敗: {e}")
    print("嘗試使用手動繪製")

In [ ]:
# 創建霸凌分類的Force Plot
print("創建Force Plot - 霸凌分類")
try:
    plt.figure(figsize=(15, 8))
    shap_visualizer.create_force_plot(
        shap_result, 
        task="bullying",
        save_path="../reports/shap_force_plot_bullying.png"
    )
    plt.show()
except Exception as e:
    print(f"Force plot創建失敗: {e}")

## 6. Waterfall Plot 可視化

In [ ]:
# 創建Waterfall Plot
print("創建Waterfall Plot - 毒性分類")
fig = shap_visualizer.create_waterfall_plot(
    shap_result, 
    task="toxicity",
    save_path="../reports/shap_waterfall_plot_toxicity.png"
)
plt.show()

In [ ]:
# 創建情緒分類的Waterfall Plot
print("創建Waterfall Plot - 情緒分類")
fig = shap_visualizer.create_waterfall_plot(
    shap_result, 
    task="emotion",
    save_path="../reports/shap_waterfall_plot_emotion.png"
)
plt.show()

## 7. Text Plot 可視化

In [ ]:
# 創建Text Plot
print("創建Text Plot - 毒性分類")
fig = shap_visualizer.create_text_plot(
    shap_result, 
    task="toxicity",
    save_path="../reports/shap_text_plot_toxicity.png"
)
plt.show()

## 8. 批量分析多個文本

In [ ]:
# 批量分析所有測試文本
print("批量分析所有測試文本...")
all_shap_results = []

for i, text in enumerate(tqdm(test_texts, desc="SHAP分析")):
    try:
        result = shap_explainer.explain_text(text, max_evals=200)  # 減少評估次數加速
        all_shap_results.append(result)
        print(f"{i+1}. {text[:20]}... -> 毒性:{result.toxicity_pred}, 霸凌:{result.bullying_pred}")
    except Exception as e:
        print(f"分析失敗 - {text[:20]}...: {e}")

print(f"\n完成 {len(all_shap_results)} 個文本的SHAP分析")

## 9. Summary Plot 全局分析

In [ ]:
# 創建Summary Plot
if len(all_shap_results) > 0:
    print("創建Summary Plot - 毒性分類")
    fig = shap_visualizer.create_summary_plot(
        all_shap_results, 
        task="toxicity",
        max_features=15,
        save_path="../reports/shap_summary_plot_toxicity.png"
    )
    plt.show()
else:
    print("沒有可用的SHAP結果用於Summary Plot")

## 總結

本筆記本展示了如何使用SHAP對中文網路霸凌偵測模型進行全面的可解釋性分析：

1. **Force Plots**: 顯示各個token對最終預測的正負貢獻
2. **Waterfall Plots**: 層次化展示特徵如何影響預測結果
3. **Text Plots**: 直觀的文本級可視化，易於理解
4. **Summary Plots**: 全局特徵重要性分析
5. **誤判分析**: 識別模型的錯誤模式
6. **方法對比**: SHAP與IG結果的對比驗證

這些可視化方法有助於：
- 理解模型的決策過程
- 識別重要的語言特徵
- 發現模型的偏見和錯誤模式
- 提高模型的可信度和可解釋性

建議在實際部署前使用這些工具對模型進行全面的可解釋性評估。